In [1]:
from PRClasses import DiscreteD, GaussD, HMM, MarkovChain
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import sounddevice as sd

from matplotlib import pyplot as plt
import numpy as np
import os

In [2]:
def GetSpeechFeatures(signal, fs, winlen, winstep, nfilt, nfft, winfunc):
    '''
    :param signal:
    :param fs:
    :param winlen:
    :param winstep:
    :param nfilt:
    :param nfft:
    :param winfunc:
    :return: the normalized MFCCs
    '''
    features_mfcc = mfcc(signal, fs,
                         winlen=winlen, winstep=winstep,
                         nfilt=nfilt,
                         nfft=nfft,
                         winfunc=winfunc)

    features_mfcc = features_mfcc.T
    features_mfcc_nor = np.zeros((features_mfcc.shape[0], features_mfcc.shape[1]))
    for i in range(features_mfcc.shape[0]):
        features_mfcc_nor[i, :] = features_mfcc[i, :] - np.mean(features_mfcc[i, :])
        features_mfcc_nor[i, :] = features_mfcc_nor[i, :] / np.std(features_mfcc_nor[i, :])

    return features_mfcc_nor

In [5]:
ad_dir = 'train_audio'
ad_list = os.listdir(ad_dir)
ad_list.sort()
ad_dir = ad_dir + '/'
num_s = np.array([6, 4, 6, 4, 4, 4, 6, 6, 4, 5])

w = 0                                       # ad_count
for ad in ad_list:

    wav_dir = os.path.join(ad_dir, ad)
    wav_list = os.listdir(wav_dir)
    # wav_list.sort()

    print('audio:', ad)

    # # 5-Fold Cross Validation
    # a = len(wav_list) - 1920
    # train_list = [[] for i in range(5)]
    # train_list[0] = wav_list[a: ]
    # train_list[1] = wav_list[0: a]
    # train_list[1].extend(wav_list[2*a: ])
    # train_list[2] = wav_list[0: 2*a]
    # train_list[2].extend(wav_list[3*a: ])
    # train_list[3] = wav_list[0: 3*a]
    # train_list[3].extend(wav_list[4*a: ])
    # train_list[4] = wav_list[0: 1920]

    # 2-Fold Cross Validation
    train_list = [[] for i in range(numf)]
    train_list[0] = wav_list[600: 1400]
    train_list[1] = wav_list[0: 800]
    numf = 2

    for tt in range(numf):

        wav_dir = wav_dir + '/'
        wav0path = os.path.join(wav_dir, train_list[tt][0])
        fs, sig_0 = wav.read(wav0path)
        t = len(sig_0) / fs


        # Initialization of q, A and state-conditional distributions
        ndim = 13
        q = np.zeros(num_s[w])
        q[0] = 1
        A = np.zeros((num_s[w], num_s[w] + 1))
        for i in range(A.shape[0]):
            A[i, i] = 0.5
            A[i, i+1] = 0.5

        meanD = np.zeros(ndim)
        covD = np.eye(ndim)

        sD = [None] * num_s[w]
        for i in range(num_s[w]):
            sD[i] = GaussD(means=meanD, cov=covD)


        # Initialization of  MarkovChain and HMM model
        mc = MarkovChain(q, A)
        h = HMM(mc, sD)


        # Training the HMM model
        nsample = len(train_list[tt])  # the number of clips used for training
        sig = np.zeros((nsample, len(sig_0)))
        num_train = 0
        mfcc_all = {}


        ii = 0
        for wavf in train_list[tt]:

            wav_path = os.path.join(wav_dir, wavf)   # The path of each wav clip
            _, sig_temp = wav.read(wav_path)
            if len(sig_temp) == sig.shape[1]:
                sig[ii] = sig_temp
                # Calculating the MFCCs of each training audio
                mfcc_test = GetSpeechFeatures(sig[ii], fs, 0.02, 0.02, 30, 320, np.hanning)
                _, px, _ = h.Get_px(mfcc_test)
                ifinf = np.isinf(px)
                ifnan = np.isnan(px)
                test_ovfl = np.count_nonzero(ifinf) + np.count_nonzero(ifnan)
                ii = ii + 1
                # If overflowed, skip this clip sample
                if test_ovfl > 0:
                    continue
                # If not overflowed, train the HMM with the current clip
                else:
                    mfcc_all[num_train] = mfcc_test
                    num_train = num_train + 1

        A_new, cov_new = h.train(mfcc_all)


        # Updating the mc and HMM model
        mc = MarkovChain(q, A_new)
        sD_new = {}
        for i in range(num_s[w]):
            sD_new[i] = GaussD(means=meanD, cov=cov_new[i])
        h = HMM(mc, sD_new)

        # Check if the covariance matrix is invertible
        _, px_new, _ = h.Get_px(mfcc_all[1])

        para = [q, A_new, sD_new]
        np.save('para0525/{}_{}'.format(ad, tt), para)

    w = w + 1

audio: down
2359
audio: go
2372
audio: left
2353
audio: no
2375
audio: off
2357
audio: on
2367
audio: right
2367
audio: stop
2380
audio: up
2375
audio: yes
2377
